In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [34]:
df_week = pd.read_csv("SAP_weekly_return_volatility.csv")
df_week.head(5)

,Year,Week_Number,mean_return,volatility,label
0,2019,0,-0.1050,3.594650,0
1,2019,1,0.6532,1.153409,1
2,2019,2,0.7326,1.198802,1
3,2019,3,0.0735,1.593999,0
4,2019,4,-0.3746,2.056743,0


In [35]:
df_daily = pd.read_csv("SAP_weekly_return_volatility_detailed.csv")
df_daily.head(5)

,High,Low,Open,Close,Volume,Adj Close,Return,Date,Week_Number,Year,Day,Weekday,mean_return,volatility
0,99.559998,98.339996,98.570000,99.180000,506300.0,94.896118,0.000,2019-01-02,0,2019,2,Wednesday,-0.1050,3.594650
1,96.830002,95.449997,96.730003,95.459999,807800.0,91.336800,-3.751,2019-01-03,0,2019,3,Thursday,-0.1050,3.594650
2,99.199997,96.910004,97.339996,98.739998,1038100.0,94.475128,3.436,2019-01-04,0,2019,4,Friday,-0.1050,3.594650
3,100.190002,98.760002,99.440002,99.709999,861600.0,95.403229,0.982,2019-01-07,1,2019,7,Monday,0.6532,1.153409
4,101.480003,100.430000,101.370003,101.269997,548600.0,96.895844,1.565,2019-01-08,1,2019,8,Tuesday,0.6532,1.153409


In [36]:
df_week_2019 = df_week[df_week["Year"] == 2019].reset_index(drop = True)
df_week_2020 = df_week[df_week["Year"] == 2020].reset_index(drop = True)

In [37]:
x_train1 = df_week_2019[["mean_return","volatility"]].values
y_train1 = df_week_2019["label"].values
x_test1 = df_week_2020[["mean_return","volatility"]].values
y_test1 = df_week_2020["label"].values

scaler = StandardScaler()
scaler.fit(x_train1)
x_s_tr = scaler.transform(x_train1)
x_s_te = scaler.transform(x_test1)

In [38]:
svm_classifier = svm.SVC(kernel = "linear")
svm_classifier.fit(x_s_tr, y_train1)
linear_predicted = svm_classifier.predict(x_s_te)
linear_accuracy = round(svm_classifier.score(x_s_te, y_test1) * 100,2)

### 1. implement a linear SVM. What is the accuracy of your SVM for year 2?

In [39]:
print(f"Accuracy for Linear SVM: {linear_accuracy}%")

Accuracy for Linear SVM: 84.91%


### 2. compute the confusion matrix for year 2

In [40]:
tn, fp, fn, tp = confusion_matrix(y_test1, linear_predicted).ravel()
pd.DataFrame([tn, fp, fn, tp], index = ["TN","FP","FN","TP"]).T

,TN,FP,FN,TP
0,29,4,4,16


### 3. what is true positive rate and true negative rate for year 2?

In [41]:
tpr = tp / (tp + fn)
tnr = tn / (tn + fp)

print(f"TPR:{tpr:.2f} and TNR:{tnr:.2f}")

TPR:0.80 and TNR:0.88


### 4. implement a Gaussian SVM and compute its accuracy for year 2? Is it better than linear SVM (use default values for parameters)

In [42]:
svm_classifier1 = svm.SVC(kernel = "rbf")
svm_classifier1.fit(x_s_tr, y_train1)
gaussian_predicted = svm_classifier1.predict(x_s_te)
gaussian_accuracy = round(svm_classifier1.score(x_s_te, y_test1) * 100,2)

print(f"Accuracy for Gaussian SVM: {gaussian_accuracy}%. Its accuracy is better then that of Linear SVM.")

Accuracy for Gaussian SVM: 86.79%. Its accuracy is better then that of Linear SVM.


### 5. implement polynomial SVM for degree 2 and compute its accuracy? Is it better than linear SVM?

In [51]:
svm_classifier2 = svm.SVC(kernel = "poly", degree = 2)
svm_classifier2.fit(x_s_tr, y_train1)
poly_predicted = svm_classifier2.predict(x_s_te)
poly_accuracy = round(svm_classifier2.score(x_s_te, y_test1) * 100,2)

print(f"Accuracy for Polynomial SVM: {poly_accuracy}%. Its far worse accuracy then that of Linear SVM.")

Accuracy for Polynomial SVM: 62.26%. Its far worse accuracy then that of Linear SVM.


### 6. implement a trading strategy based on your labels (from linear SVM) for year 2 and compare the performance with the ”buy-and-hold” strategy. Which strategy results in a larger amount at the end of the year?

In [44]:
df_week_2020["predict"] = linear_predicted
df_daily_2 = df_daily[(df_daily["Year"] == 2020)].copy().reset_index(drop=True)

#### Buy and Hold

In [45]:
def buyhold(ret_list):
    ret = round(((ret_list / 100) + 1).cumprod() * 100,2)
    
    return ret

In [46]:
hold = buyhold(df_daily[(df_daily["Year"] == 2020)]["Return"])
hold.values[-1]

98.71

#### My Strategy

In [47]:
def compute_daily_ret(d_frame1, amt):
    ret = round(((d_frame1["Return"] / 100) + 1).cumprod() * amt,2)

    return round(ret.values[-1],2)

In [48]:
def weekDaily(d_frame_week,d_frame_daily,lbl):
    total_ret = list()
    for i in range(len(d_frame_week)):
        if i > 0:
            year = d_frame_week.loc[i]["Year"]

            week = d_frame_week.loc[i]["Week_Number"]

            prv_week = d_frame_week.loc[i - 1]["Week_Number"]

            frame = d_frame_daily[(d_frame_daily["Week_Number"] == week)]

            if d_frame_week.loc[i - 1][lbl] == 0 and d_frame_week.loc[i][lbl] == 1:

                amount = total_ret[i - 1] 
                total_ret.append(compute_daily_ret(frame, amount))
                
            elif d_frame_week.loc[i - 1][lbl] == 1 and d_frame_week.loc[i][lbl] == 1:

                amount = total_ret[i - 1]
                total_ret.append(compute_daily_ret(frame, amount))
            else:
                total_ret.append(total_ret[i - 1])
        else:     
            total_ret.append(100)
    return total_ret

In [49]:
daily_ret = weekDaily(df_week_2020,df_daily_2,"predict")
daily_ret[-1]

284.75

In [50]:
print(f"""
2020 return using Buy and Hold: $ {hold.values[-1]} and 
2020 return using Random Forest: $ {daily_ret[-1]}
""")


2020 return using Buy and Hold: $ 98.71 and 
2020 return using Random Forest: $ 284.75

